# Borovickova-Shimer example

In [1]:
# Add PyTwoWay to system path (do not run this)
# import sys
# sys.path.append('../../..')

## Import the PyTwoWay package

Make sure to install it using `pip install pytwoway`.

In [2]:
from pandas import Series
import pytwoway as tw
import bipartitepandas as bpd

## First, check out parameter options

Do this by running:

- Cleaning - `bpd.clean_params().describe_all()`

- Simulating - `tw.sim_bs_params().describe_all()`

Alternatively, run `x_params().keys()` to view all the keys for a parameter dictionary, then `x_params().describe(key)` to get a description for a single key.

## Second, set parameter choices

<div class="alert alert-info">

Note

We set `copy=False` in `clean_params` to avoid unnecessary copies (although this may modify the original dataframe).

</div>

In [3]:
# Cleaning
clean_params = bpd.clean_params(
    {
        'connectedness': 'strongly_connected',
        'drop_single_stayers': True,
        'drop_returns': 'returners',
        'copy': False
    }
)
# Simulating
sim_params = tw.sim_bs_params(
    {
        'n_workers': 1000,
        'n_firms': 100,
        'sigma_lambda': 1.25,
        'sigma_mu': 0.75,
        'sigma_wages': 2.5,
        'rho': -0.5
    }
)

## Third, extract data (we simulate for the example)

`PyTwoWay` contains the class `SimBS` which we use here to simulate from the Borovickova-Shimer dgp. If you have your own data, you can import it during this step. Load it as a `Pandas DataFrame` and then convert it into a `BipartitePandas DataFrame` in the next step.

In [4]:
sim_data = tw.SimBS(sim_params).simulate()

## Fourth, prepare data

This is exactly how you should prepare real data prior to running the Borovickova-Shimer estimator.

- First, we convert the data into a `BipartitePandas DataFrame`

- Second, we clean the data (e.g. drop NaN observations, make sure firm and worker ids are contiguous, construct the strongly connected set, etc.)

- Third, we collapse the data at the worker-firm spell level (take mean wage over the spell)

Further details on `BipartitePandas` can be found in the package documentation, available [here](https://tlamadon.github.io/bipartitepandas/).

In [5]:
# Convert into BipartitePandas DataFrame
bdf = bpd.BipartiteDataFrame(sim_data)
# Clean
bdf = bdf.clean(clean_params)
# Collapse
bdf = bdf.collapse(is_sorted=True, copy=False)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how='returners')
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how='strongly_connected')
sorting columns
resetting index


## Fifth, initialize and run the estimator

<div class="alert alert-info">

Note

We can also estimate the alternative estimator by specifying `alternative_estimator=True`.

</div>

In [6]:
# Initialize Borovickova-Shimer estimator
bs_estimator = tw.BSEstimator()
# Fit Borovickova-Shimer estimator
bs_estimator.fit(bdf, alternative_estimator=False)

## Finally, investigate the results

Estimated firm values are stored in the class attribute `.res`.

In [7]:
bs_estimator.res

{'y_bar': 0.018035566471320305,
 'sigma_sq_lambda': 1.5046638241648331,
 'sigma_sq_mu': 0.41955038465163036,
 'cov(lambda, mu)': -0.42242174619834055,
 'corr(lambda, mu)': -0.5316605863572214}